In [8]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, isnull, udf, when

from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import datetime

import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt


In [1]:
# Create spark session
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1502a5c9-08b3-41f8-8c23-3f1705d3e879',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'GxPOyfly2xFRbHdmJ-Ekd-oqsGzrsV_wo_uO2f9th8Nx'
}

configuration_name = 'os_5af7ef365f01481399b986af65e70d88_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-aovkpzlqasvjfe'))
df_data_1.take(5)


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200314214151-0000
KERNEL_ID = 9152b68e-3fa6-4c3c-987c-0dcd0d57a93a


[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', 

In [3]:
# clean up: remove all rows where the userId is empty as these are from events where a user was 
# not logged in which is non-relevant for our churn analysis

df_clean = df_data_1.filter(df_data_1.userId != "")

In [4]:
df_clean.describe("userId", "sessionId").show()

+-------+------------------+------------------+
|summary|            userId|         sessionId|
+-------+------------------+------------------+
|  count|            528005|            528005|
|   mean| 60268.42669103512|2042.9801820058522|
| stddev|109898.82324176628|1433.9981489410682|
|    min|                10|                 1|
|    max|                99|              4808|
+-------+------------------+------------------+



In [5]:
df_clean.count()

528005

In [9]:
# Add the hour of day when the song was played to the data frame as a feature
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)
df_clean = df_clean.withColumn("hour", get_hour(df_clean.ts))

In [10]:
# Filter the data frame to only contain churn events
df_churn = df_clean.where(df_clean.auth=="Cancelled")

In [11]:
# create a churn column in the dataframe that is 1 for all the events that are from users that have churned
list_churnusers = [(row['userId']) for row in df_churn.collect()]

df_clean_churn = df_clean.withColumn("churn", when(col("userId").isin(list_churnusers),1).otherwise(0))

In [12]:
# Now its time create the features for each userId
# Feature 1: Gender
f1 = df_clean_churn.select('userId', 'gender').dropDuplicates().replace(['F', 'M'], ['0', '1'], 'gender').select('userId', col('gender').cast('int'))

f1.show()

+------+------+
|userId|gender|
+------+------+
|   188|     0|
|   234|     1|
|    44|     0|
|   273|     1|
|    46|     0|
|   220|     0|
|    41|     0|
|   186|     1|
|   254|     1|
|   280|     1|
|100035|     0|
|   294|     0|
|    72|     0|
|300023|     0|
|    39|     0|
|   287|     1|
|100010|     0|
|200026|     1|
|   210|     0|
|   207|     1|
+------+------+
only showing top 20 rows



In [13]:
# Feature 2: Level
f2 = df_clean_churn.select('userId', 'level').dropDuplicates().replace(['free', 'paid'], ['0', '1'], 'level').select('userId', col('level').cast('int'))

f2.show()

+------+-----+
|userId|level|
+------+-----+
|100021|    0|
|100029|    0|
|    94|    1|
|200050|    1|
|    18|    0|
|   185|    0|
|200021|    0|
|   241|    1|
|200001|    0|
|   207|    0|
|     6|    0|
|   190|    1|
|   227|    0|
|   289|    1|
|300022|    1|
|   168|    1|
|   112|    0|
|100035|    1|
|   132|    0|
|   165|    0|
+------+-----+
only showing top 20 rows



In [14]:
# Feature 3: Number of songs listened per session
f3  = df_clean_churn.where(df_clean_churn.page == "NextSong").groupby(['userId', 'sessionId']).count().groupby('userId').agg({'count' : 'avg'}).withColumnRenamed('avg(count)', 'songs_per_session')

f3.show()

+------+------------------+
|userId| songs_per_session|
+------+------------------+
|200002|              62.0|
|100010|              48.0|
|   296|              22.4|
|   125|20.666666666666668|
|   124|           114.125|
|     7|              30.8|
|    51|              53.2|
|200037|17.428571428571427|
|   205|             209.6|
|   169|              14.0|
|   272|              48.5|
|    15|101.94444444444444|
|   234| 72.78260869565217|
|   232| 81.56521739130434|
|   282| 93.89285714285714|
|    54|            64.625|
|   155|              35.0|
|200043|             51.25|
|   154|             23.25|
|   132|              56.5|
+------+------------------+
only showing top 20 rows



In [15]:
# Feature 4 : Number of songs played / total number of events per user
f4 = df_clean_churn.select('userId', 'song').groupBy('userId').count().withColumnRenamed('count', 'total_songs')
f4.show()

+------+-----------+
|userId|total_songs|
+------+-----------+
|100010|        137|
|200002|        395|
|   296|        152|
|   125|         84|
|   124|       2156|
|     7|        600|
|    51|        328|
|200037|        155|
|   205|       1244|
|   169|        149|
|   272|        131|
|   282|       3191|
|   232|       2325|
|    15|       2173|
|    54|        624|
|   234|       2021|
|200043|        808|
|   155|        172|
|   154|        126|
|   132|        144|
+------+-----------+
only showing top 20 rows



In [16]:
# Feature 5: Number of songs added to playlists
f5 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Add to Playlist').groupBy('userId').count().withColumnRenamed('count', 'playlist_adds')
f5.show()

+------+-------------+
|userId|playlist_adds|
+------+-------------+
|100010|            1|
|200002|            6|
|   296|            3|
|   125|            2|
|    51|            8|
|   124|           45|
|     7|            6|
|200037|            4|
|   205|           25|
|   272|            3|
|   282|           66|
|   232|           61|
|   234|           51|
|    54|           15|
|    15|           57|
|   155|            4|
|200043|           23|
|100014|           11|
|   154|            4|
|   132|            4|
+------+-------------+
only showing top 20 rows



In [18]:
# Feature 6: Number of friends added
f6 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Add Friend').groupBy('userId').count().withColumnRenamed('count', 'friends_added')
f6.show()

+------+-------------+
|userId|friends_added|
+------+-------------+
|   296|            2|
|100010|            3|
|200002|            2|
|   125|            3|
|   124|           26|
|     7|           16|
|    51|            3|
|200037|            2|
|   205|           21|
|   169|            6|
|   282|           79|
|   232|           43|
|    15|           32|
|    54|           20|
|   234|           17|
|200043|           12|
|   154|            4|
|   132|            4|
|100014|            3|
|300027|            4|
+------+-------------+
only showing top 20 rows



In [19]:
# Feature 7: Number of errors
f7 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Error').groupBy('userId').count().withColumnRenamed('count', 'errors')
f7.show()

+------+------+
|userId|errors|
+------+------+
|    51|     2|
|   205|     1|
|   282|     4|
|   232|     1|
|    15|     1|
|   234|     2|
|   155|     1|
|   101|     6|
|   279|     3|
|300017|     4|
|   138|     1|
|    29|     5|
|    69|     1|
|    42|     4|
|300033|     1|
|    87|     5|
|    73|     1|
|300035|     5|
|    30|     7|
|   113|     2|
+------+------+
only showing top 20 rows



In [20]:
# Feature 8: Number of adverts
f8 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Roll Advert').groupBy('userId').count().withColumnRenamed('count', 'adverts')
f8.show()

+------+-------+
|userId|adverts|
+------+-------+
|   296|      9|
|100010|     22|
|200002|     11|
|   125|      6|
|     7|     35|
|    51|      1|
|   124|      1|
|200037|      9|
|   169|     13|
|   272|     12|
|   282|     37|
|   232|     38|
|   234|     30|
|200043|     41|
|   154|      9|
|   132|      5|
|100014|      3|
|300027|     26|
|   101|      6|
|    11|     10|
+------+-------+
only showing top 20 rows



In [21]:
# Feature 9: Number of thumbs up
f9 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Thumbs Up').groupBy('userId').count().withColumnRenamed('count', 'thumbs_up')
f9.show()

+------+---------+
|userId|thumbs_up|
+------+---------+
|100010|        4|
|200002|       15|
|   296|        8|
|   125|        3|
|   124|      102|
|    51|       16|
|     7|       12|
|200037|        5|
|   205|       50|
|   169|        6|
|   272|        7|
|   282|      133|
|   232|       97|
|    15|       93|
|    54|       26|
|   234|       91|
|   155|        7|
|200043|       33|
|100014|       14|
|   154|        4|
+------+---------+
only showing top 20 rows



In [22]:
# Feature 10: Number of thumbs down
f10 = df_clean_churn.select('userId', 'page').where(df_clean_churn.page == 'Thumbs Down').groupBy('userId').count().withColumnRenamed('count', 'thumbs_down')
f10.show()

+------+-----------+
|userId|thumbs_down|
+------+-----------+
|100010|          3|
|200002|          5|
|   125|          1|
|   124|         15|
|    51|          1|
|     7|          4|
|200037|          3|
|   205|         10|
|   272|          4|
|   282|         29|
|   232|         17|
|    15|         16|
|    54|          7|
|   234|         22|
|200043|         22|
|   154|          2|
|   132|          1|
|100014|          3|
|300027|          5|
|   200|          2|
+------+-----------+
only showing top 20 rows



In [23]:
churn = df_clean_churn.select('userId', col('churn').alias('churn')).dropDuplicates()
churn.show()

+------+-----+
|userId|churn|
+------+-----+
|    19|    0|
|    73|    0|
|   209|    0|
|100005|    1|
|100030|    1|
|100034|    0|
|200007|    0|
|   184|    0|
|    50|    0|
|100048|    1|
|    30|    0|
|100011|    1|
|100012|    1|
|300051|    0|
|   172|    1|
|     8|    0|
|   192|    0|
|   211|    0|
|     4|    0|
|200018|    0|
+------+-----+
only showing top 20 rows



In [24]:
# as a final step we combine all the features with the churn column into a feature_data dataframe

feature_data = f1.join(f2, 'userId', 'inner').join(f3, 'userId', 'inner').join(f4, 'userId', 'inner')\
             .join(f5, 'userId', 'inner').join(f6, 'userId', 'inner').join(f7, 'userId', 'inner')\
             .join(f8, 'userId', 'inner').join(f9, 'userId', 'inner').join(f10, 'userId', 'inner').join(churn, 'userId', 'inner').drop('userId')

feature_data.show()

+------+-----+------------------+-----------+-------------+-------------+------+-------+---------+-----------+-----+
|gender|level| songs_per_session|total_songs|playlist_adds|friends_added|errors|adverts|thumbs_up|thumbs_down|churn|
+------+-----+------------------+-----------+-------------+-------------+------+-------+---------+-----------+-----+
|     1|    1|              53.2|        328|            8|            3|     2|      1|       16|          1|    1|
|     0|    0| 81.56521739130434|       2325|           61|           43|     1|     38|       97|         17|    0|
|     0|    1| 81.56521739130434|       2325|           61|           43|     1|     38|       97|         17|    0|
|     1|    1| 72.78260869565217|       2021|           51|           17|     2|     30|       91|         22|    1|
|     1|    0| 72.78260869565217|       2021|           51|           17|     2|     30|       91|         22|    1|
|     1|    0| 93.89285714285714|       3191|           66|     

In [25]:
# before we start the modelling we need to combine the features into a features vector
feature_columns = ['gender', 'level', 'songs_per_session', 'total_songs', 'playlist_adds', 'friends_added', 'errors', 'adverts', 'thumbs_up', 'thumbs_down']
assmbler = VectorAssembler(inputCols = feature_columns, outputCol = "vectorized_features")
feature_data = assmbler.transform(feature_data)
feature_data.show()

+------+-----+------------------+-----------+-------------+-------------+------+-------+---------+-----------+-----+--------------------+
|gender|level| songs_per_session|total_songs|playlist_adds|friends_added|errors|adverts|thumbs_up|thumbs_down|churn| vectorized_features|
+------+-----+------------------+-----------+-------------+-------------+------+-------+---------+-----------+-----+--------------------+
|     1|    1|              53.2|        328|            8|            3|     2|      1|       16|          1|    1|[1.0,1.0,53.2,328...|
|     0|    0| 81.56521739130434|       2325|           61|           43|     1|     38|       97|         17|    0|[0.0,0.0,81.56521...|
|     0|    1| 81.56521739130434|       2325|           61|           43|     1|     38|       97|         17|    0|[0.0,1.0,81.56521...|
|     1|    1| 72.78260869565217|       2021|           51|           17|     2|     30|       91|         22|    1|[1.0,1.0,72.78260...|
|     1|    0| 72.78260869565217| 

In [26]:
# in order to enable our machine learning models to work with our data without getting bias though different scales
# we need to scale the data
scaler = StandardScaler(inputCol = 'vectorized_features', outputCol = "scaled_features", withStd = True, withMean = False)
scaler_model = scaler.fit(feature_data)
feature_data = scaler_model.transform(feature_data)

In [27]:
# here we prepare the data into label and features vector as well was renaming these to use the standard 
# column names used by the used classifiers.
feature_data_final = feature_data.select(['churn', 'scaled_features'])
feature_data_final = feature_data_final.selectExpr("churn as label", "scaled_features as features")

In [28]:
# splitting the data in training and testing data sets.
train_data , test_data = feature_data_final.randomSplit([0.7,0.3], seed = 42)

In [29]:
# Fitting a Random Forest Classifier
rfc = RandomForestClassifier()

mce_f1 = MulticlassClassificationEvaluator(metricName = 'f1')
parameter_grid = ParamGridBuilder().build()

cv_rf = CrossValidator(estimator = rfc, estimatorParamMaps = parameter_grid, evaluator = mce_f1, numFolds = 2)

cv_rf_model = cv_rf.fit(train_data)
cv_rf_model.avgMetrics

[0.7845075906022849]

In [30]:
# Evaluating the results with regard to f1 score and accuracy
test_result_rf = cv_rf_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction")
evaluator_ROC = BinaryClassificationEvaluator(rawPredictionCol='prediction', metricName='areaUnderROC')
print("f1 score: " + str(evaluator.evaluate(test_result_rf, {evaluator.metricName : "accuracy"})))
print("accuracy: " + str(evaluator.evaluate(test_result_rf, {evaluator.metricName : "f1"})))
print("area under ROC: " + str(evaluator_ROC.evaluate(test_result_rf, {evaluator.metricName : "areaUnderROC"})))

f1 score: 0.8198198198198198
accuracy: 0.7691441441441441
area under ROC: 0.6


In [31]:
# Fitting a Gradient Boost Tree Classifier
gbtc = GBTClassifier(maxIter = 5, maxDepth = 5, seed = 42)

cv_gbtc = CrossValidator(estimator = gbtc, estimatorParamMaps = parameter_grid, evaluator = mce_f1, numFolds = 2)

cv_gbtc_model = cv_gbtc.fit(train_data)
cv_gbtc_model.avgMetrics

[0.7306828246070239]

In [32]:
# Evaluating the results with regard to f1 score and accuracy
test_result_gbtc = cv_gbtc_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction")
evaluator_ROC = BinaryClassificationEvaluator(rawPredictionCol='prediction', metricName='areaUnderROC')
print("f1 score: " + str(evaluator.evaluate(test_result_gbtc, {evaluator.metricName : "accuracy"})))
print("accuracy: " + str(evaluator.evaluate(test_result_gbtc, {evaluator.metricName : "f1"})))
print("area under ROC: " + str(evaluator_ROC.evaluate(test_result_gbtc, {evaluator.metricName : "areaUnderROC"})))

f1 score: 0.8918918918918919
accuracy: 0.8902609506057781
area under ROC: 0.8309302325581395


In [34]:
# Fitting a Gradient Boost Tree Classifier and using Grid Search to find the optimal hyperparameters
gbtc_final = GBTClassifier(maxIter = 5, maxDepth = 5, seed = 42)
parameter_grid_final = ParamGridBuilder().addGrid(gbtc_final.maxIter, [5, 10, 15, 20]).addGrid(gbtc_final.maxDepth, [5, 10]).build()

cv_gbtc_final = CrossValidator(estimator = gbtc_final, estimatorParamMaps = parameter_grid_final, evaluator = mce_f1, numFolds = 3)
cv_gbtc_model_final = cv_gbtc_final.fit(train_data)
cv_gbtc_model_final.avgMetrics

[0.8224143563442641,
 0.7984631218726905,
 0.8203306263120651,
 0.8029130734028762,
 0.8172722614559473,
 0.8029130734028762,
 0.8137199487746023,
 0.8029130734028762]

In [35]:
# Evaluating the results with regard to f1 score and accuracy
test_result_gbtc_final = cv_gbtc_model_final.transform(test_data)

evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction")
evaluator_ROC = BinaryClassificationEvaluator(rawPredictionCol='prediction', metricName='areaUnderROC')

print("f1 score: " + str(evaluator.evaluate(test_result_gbtc_final, {evaluator.metricName : "accuracy"})))
print("accuracy: " + str(evaluator.evaluate(test_result_gbtc_final, {evaluator.metricName : "f1"})))
print("area under ROC: " + str(evaluator_ROC.evaluate(test_result_gbtc_final, {evaluator.metricName : "areaUnderROC"})))

f1 score: 0.8918918918918919
accuracy: 0.8902609506057781
area under ROC: 0.8309302325581395
